# Реализация метода

## База

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def mixed_norm_polyfit(x, y, c1, c2, degree):
    """
    Поиск коэффициентов полинома, минимизирующего линейную комбинацию L1 и L2 ошибок.
    
    Потери для одной точки: c1 * |F(x_i) - y_i| + c2 * (F(x_i) - y_i)^2
    Итоговые потери: сумма по всем точкам.
    
    Аргументы:
        x : array-like, форма (n_samples,)
            Входные значения
        y : array-like, форма (n_samples,)
            Целевые значения
        c1 : float
            Коэффициент для L1-члена (должен быть >= 0)
        c2 : float
            Коэффициент для L2-члена (должен быть >= 0)
        degree : int
            Степень полинома
    
    Возвращает:
        coefficients : ndarray, форма (degree + 1,)
            Коэффициенты полинома [a0, a1, ..., a_degree] для F(x) = a0 + a1*x + ... + a_degree*x^degree
        y_pred : ndarray, форма (n_samples,)
            Предсказанные значения на входных x
    """
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    
    if x.shape[0] == 0 or y.shape[0] == 0:
        return np.zeros(degree + 1), np.array([])
    
    if x.shape[0] != y.shape[0]:
        raise ValueError("Длины x и y должны совпадать")
    
    if c1 < 0 or c2 < 0:
        raise ValueError("Коэффициенты c1 и c2 должны быть неотрицательными")
    
    if c2 == 0:
        c2 = 1e-15
    
    # Создаём матрицу Вандермонда: столбцы [x^0, x^1, ..., x^degree]
    X = np.vander(x, N=degree + 1, increasing=True)
    
    # Начальное приближение через МНК (устойчиво даже для недоопределённых систем)
    try:
        coef0, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    except np.linalg.LinAlgError:
        coef0 = np.zeros(degree + 1)
    
    # Вырожденный случай: нулевые веса
    if c1 == 0 and c2 == 0:
        return coef0, X @ coef0
    
    # Целевая функция с защитой от переполнения
    def loss(coef):
        residuals = X @ coef - y
        # Для устойчивости: избегаем возведения в квадрат очень больших чисел
        if c2 > 0 and np.any(np.abs(residuals) > 1e10):
            return np.inf
        l1_term = np.sum(np.abs(residuals))
        l2_term = np.sum(residuals ** 2) if c2 > 0 else 0.0
        return c1 * l1_term + c2 * l2_term
    
    # Основная оптимизация: метод Пауэлла (хорошо работает с негладкими функциями)
    result = minimize(
        loss,
        coef0,
        method='Powell',
        options={'maxiter': 10000, 'disp': False}
    )
    
    # Резервный метод при неудаче
    if not result.success:
        result = minimize(
            loss,
            coef0,
            method='Nelder-Mead',
            options={'maxiter': 10000, 'disp': False}
        )
    
    coefficients = result.x
    y_pred = X @ coefficients
    
    return coefficients, y_pred

In [5]:
x = np.array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4])
y = np.array([1, 3, 7, 13, 22, 1, 3, 7, 14, 22, 1, 3, 7, 13, 22, 1, 3, 7, 14, 22, 1, 3, 7, 13, 22, 1, 3, 7, 14, 22])  # Соответствует y = x² + x + 1

# Подгонка квадратичного полинома с акцентом на устойчивость к выбросам (L1)
coef, y_pred = mixed_norm_polyfit(x, y, c1=1.0, c2=1, degree=2)
print("Коэффициенты:", coef)  # Ожидаемый результат: [~1, ~1, ~1]
print("Предсказания:", y_pred)

Коэффициенты: [0.99999991 0.82142859 1.10714286]
Предсказания: [ 0.99999991  2.92857136  7.07142852 13.4285714  22.          0.99999991
  2.92857136  7.07142852 13.4285714  22.          0.99999991  2.92857136
  7.07142852 13.4285714  22.          0.99999991  2.92857136  7.07142852
 13.4285714  22.          0.99999991  2.92857136  7.07142852 13.4285714
 22.          0.99999991  2.92857136  7.07142852 13.4285714  22.        ]


## Подбор c1, с2 и степени бутстрапом

In [12]:
def full_grid_bootstrap_selection(x, y, degrees=(1, 2, 3, 4, 5, 6),
                                  B=200, random_state=42):
    """
    Полный .632+ бутстрап по сетке всех комбинаций (степень, c1, c2).
    
    Алгоритм:
    1. Для каждой комбинации (степень, c1, c2) из сетки:
       - Запускаем .632+ бутстрап
       - Получаем оценку истинной ошибки
    2. Выбираем комбинацию с минимальной .632+ ошибкой
    
    Это обеспечивает корректный выбор модели без утечки данных.
    """
    np.random.seed(random_state)
    
    # Сетка соотношений (c1, c2)
    ratios = [(1, 0), (0, 1), (1, 1), (3, 4), (4, 3), 
              (1, 2), (2, 1), (1, 4), (4, 1)]
    grid = [(a / (a + b), b / (a + b)) for a, b in ratios]
    
    n = len(x)
    results = {}
    
    total_combinations = len(degrees) * len(grid)
    current = 0
    
    for degree in degrees:
        for c1, c2 in grid:
            current += 1
            # print(f"[{current}/{total_combinations}] Степень={degree}, c1={c1:.2f}, c2={c2:.2f}...", 
            #       end=' ', flush=True)
            
            err_train_list = []
            err_boot_list = []
            gamma_list = []
            coefs_list = []
            
            for b in range(B):
                # 1. Бутстрэп-выборка
                indices = np.random.choice(n, size=n, replace=True)
                x_boot = x[indices]
                y_boot = y[indices]
                out_of_bag = np.setdiff1d(np.arange(n), indices)
                
                # 2. Обучаем модель с ФИКСИРОВАННЫМИ (c1, c2)
                coef, y_pred_boot = mixed_norm_polyfit(x_boot, y_boot, c1, c2, degree)
                
                # 3. Ошибка на обучении
                err_train = np.mean((y_boot - y_pred_boot) ** 2)
                
                # 4. Ошибка на выброшенных точках
                if len(out_of_bag) > 0:
                    X_oob = np.vander(x[out_of_bag], N=degree + 1, increasing=True)
                    y_pred_oob = X_oob @ coef
                    err_boot = np.mean((y[out_of_bag] - y_pred_oob) ** 2)
                else:
                    err_boot = err_train
                
                # 5. Оптимистическая ошибка γ для ЭТОЙ модели
                X_full = np.vander(x, N=degree + 1, increasing=True)
                y_pred_full = X_full @ coef
                
                y_pred_matrix = np.tile(y_pred_full, (n, 1))
                y_true_matrix = np.tile(y.reshape(-1, 1), (1, n))
                gamma = np.mean((y_true_matrix - y_pred_matrix.T) ** 2)
                
                # 6. Сохраняем
                err_train_list.append(err_train)
                err_boot_list.append(err_boot)
                gamma_list.append(gamma)
                # coefs_list.append(coef)
            
            # 7. Усредняем по итерациям
            err_train_avg = np.mean(err_train_list)
            err_boot_avg = np.mean(err_boot_list)
            gamma_avg = np.mean(gamma_list)
            
            # 8. .632+ формула
            R = (err_boot_avg - err_train_avg) / (gamma_avg - err_train_avg + 1e-10)
            R = np.clip(R, 0, 1)
            
            w = 0.632 / (1 - 0.368 * R)
            w = np.clip(w, 0.632, 1.0)
            
            err_632plus = (1 - w) * err_train_avg + w * err_boot_avg
            
            # 9. Сохраняем результаты
            results[(degree, c1, c2)] = {
                'err_632plus': err_632plus,
                'err_train': err_train_avg,
                'err_boot': err_boot_avg,
                'gamma': gamma_avg,
                'R': R,
                'w': w
                # 'coefs': np.median(coefs_list, axis=0)  # Медианные коэффициенты
            }
            
            # print(f"✓ err_632+={err_632plus:.4f}")
    
    # 10. Выбираем лучшую комбинацию
    best_key = min(results.keys(), key=lambda k: results[k]['err_632plus'])
    best_degree, best_c1, best_c2 = best_key
    
    return best_degree, best_c1, best_c2

In [8]:
full_grid_bootstrap_selection(x, y)

[1/54] Степень=1, c1=1.00, c2=0.00... ✓ err_632+=6.3802
[2/54] Степень=1, c1=0.00, c2=1.00... ✓ err_632+=4.5330
[3/54] Степень=1, c1=0.50, c2=0.50... ✓ err_632+=4.6776
[4/54] Степень=1, c1=0.43, c2=0.57... ✓ err_632+=4.7336
[5/54] Степень=1, c1=0.57, c2=0.43... ✓ err_632+=4.8599
[6/54] Степень=1, c1=0.33, c2=0.67... ✓ err_632+=4.5013
[7/54] Степень=1, c1=0.67, c2=0.33... ✓ err_632+=4.7302
[8/54] Степень=1, c1=0.20, c2=0.80... ✓ err_632+=4.3282
[9/54] Степень=1, c1=0.80, c2=0.20... ✓ err_632+=5.0742
[10/54] Степень=2, c1=1.00, c2=0.00... ✓ err_632+=0.0583
[11/54] Степень=2, c1=0.00, c2=1.00... ✓ err_632+=0.0656
[12/54] Степень=2, c1=0.50, c2=0.50... ✓ err_632+=0.0603
[13/54] Степень=2, c1=0.43, c2=0.57... ✓ err_632+=0.0590
[14/54] Степень=2, c1=0.57, c2=0.43... ✓ err_632+=0.0600
[15/54] Степень=2, c1=0.33, c2=0.67... ✓ err_632+=0.0572
[16/54] Степень=2, c1=0.67, c2=0.33... ✓ err_632+=0.0591
[17/54] Степень=2, c1=0.20, c2=0.80... ✓ err_632+=0.0618
[18/54] Степень=2, c1=0.80, c2=0.20... ✓

(2,
 0.3333333333333333,
 0.6666666666666666,
 {(1, 1.0, 0.0): {'err_632plus': np.float64(6.380216220223122),
   'err_train': np.float64(4.138920863746104),
   'err_boot': np.float64(6.380216220223122),
   'gamma': np.float64(5.235257836101773),
   'R': np.float64(1.0),
   'w': np.float64(1.0)},
  (1, 0.0, 1.0): {'err_632plus': np.float64(4.533010642996561),
   'err_train': np.float64(3.2166931495141013),
   'err_boot': np.float64(4.533010642996561),
   'gamma': np.float64(3.8253304013485523),
   'R': np.float64(1.0),
   'w': np.float64(1.0)},
  (1, 0.5, 0.5): {'err_632plus': np.float64(4.677643127218143),
   'err_train': np.float64(3.207912997875125),
   'err_boot': np.float64(4.677643127218143),
   'gamma': np.float64(3.898708471613152),
   'R': np.float64(1.0),
   'w': np.float64(1.0)},
  (1,
   0.42857142857142855,
   0.5714285714285714): {'err_632plus': np.float64(4.73355520921433), 'err_train': np.float64(3.2114448903727713), 'err_boot': np.float64(4.73355520921433), 'gamma': np.

# Оценка

## Метрики

### Интегральные

In [10]:
from scipy.integrate import quad
from scipy.optimize import minimize_scalar, minimize

def imse(g_hat, g_true, a=0, b=1, epsabs=1e-6):
    integrand = lambda x: (g_hat(x) - g_true(x)) ** 2
    integral, _ = quad(integrand, a, b, epsabs=epsabs, limit=100)
    return integral / (b - a)

def imae(g_hat, g_true, a=0, b=1, epsabs=1e-6):
    integrand = lambda x: abs(g_hat(x) - g_true(x))
    integral, _ = quad(integrand, a, b, epsabs=epsabs, limit=100)
    return integral / (b - a)

def maxerr(g_hat, g_true, a=0, b=1, xatol=1e-6):
    objective = lambda x: -abs(g_hat(x) - g_true(x))
    result = minimize_scalar(objective, bounds=(a, b), method='bounded', options={'xatol': xatol})
    if result.success:
        return -result.fun
    x_grid = np.linspace(a, b, 500)
    return max(abs(g_hat(x) - g_true(x)) for x in x_grid)


## Тестирование на полиномиальных данных

In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Загрузка лога синтетических данных
log_df = pd.read_csv('../datasets/synthetic/synthetic_datasets_with_coeffs/seed_log.csv')
results_metrics = []
results_params = []

# Сетка для тестирования
degrees_true = range(1, 7)
noise_levels = ['low', 'moderate', 'high']
n_seeds = 30
B = 100  # бутстрэп-итераций

# Общий прогресс-бар
total_iterations = len(degrees_true) * len(noise_levels) * n_seeds
pbar = tqdm(total=total_iterations, desc="Прогресс", ncols=100)

for degree_true in degrees_true:
    for noise_level in noise_levels:
        # Загрузка датасета
        df = pd.read_csv(f'../datasets/synthetic/synthetic_datasets_with_coeffs/noise_{noise_level}_deg{degree_true}.csv')
        
        seeds = df['seed'].unique()[:n_seeds]
        
        for seed in seeds:
            subset = df[df['seed'] == seed]
            x_train = subset['x'].values
            y_train = subset['y_noisy'].values
            
            # Истинные коэффициенты
            coeffs_true = [subset[f'coeff_{i}'].iloc[0] for i in range(degree_true + 1)]
            g_true = lambda x, c=coeffs_true: sum(c[i] * x**i for i in range(len(c)))
            
            # Подбор модели через .632+ бутстрап
            best_degree, best_c1, best_c2 = full_grid_bootstrap_selection(
                x_train, y_train, 
                degrees=(1, 2, 3, 4, 5, 6),
                B=B,
                random_state=int(seed)
            )
            
            # Оценка коэффициентов с найденными параметрами
            coef_est, y_pred = mixed_norm_polyfit(x_train, y_train, best_c1, best_c2, best_degree)
            g_est = lambda x, c=coef_est: sum(c[i] * x**i for i in range(len(c)))
            
            # Расчёт метрик
            imse_val = imse(g_est, g_true)
            imae_val = imae(g_est, g_true)
            maxerr_val = maxerr(g_est, g_true)
            
            # Сохранение метрик (для агрегации)
            results_metrics.append({
                'true_degree': degree_true,
                'noise_level': noise_level,
                'seed': seed,
                'imse': imse_val,
                'imae': imae_val,
                'maxerr': maxerr_val
            })
            
            # Сохранение параметров модели (для отдельного анализа)
            results_params.append({
                'true_degree': degree_true,
                'noise_level': noise_level,
                'seed': seed,
                'selected_degree': best_degree,
                'c1': best_c1,
                'c2': best_c2,
                'coefficients': coef_est.tolist()
            })
            
            pbar.update(1)
            pbar.set_postfix({
                'deg': degree_true,
                'noise': noise_level,
                'sel_deg': best_degree
            })
        print('------------------------------------------------------------------')

pbar.close()

# Агрегация метрик
results_agg = []
for degree_true in degrees_true:
    for noise_level in noise_levels:
        mask = [(r['true_degree'] == degree_true) and (r['noise_level'] == noise_level) 
                for r in results_metrics]
        subset = [r for r, m in zip(results_metrics, mask) if m]
        
        if subset:
            imse_vals = [r['imse'] for r in subset]
            imae_vals = [r['imae'] for r in subset]
            maxerr_vals = [r['maxerr'] for r in subset]
            
            results_agg.append({
                'true_degree': degree_true,
                'noise_level': noise_level,
                'imse_mean': np.mean(imse_vals),
                'imse_sem': np.std(imse_vals) / np.sqrt(len(imse_vals)),
                'imae_mean': np.mean(imae_vals),
                'imae_sem': np.std(imae_vals) / np.sqrt(len(imae_vals)),
                'maxerr_mean': np.mean(maxerr_vals),
                'maxerr_sem': np.std(maxerr_vals) / np.sqrt(len(maxerr_vals))
            })

# Сохранение результатов
results_df = pd.DataFrame(results_agg)
results_df.to_csv('polynomial_bootstrap_results.csv', index=False)

params_df = pd.DataFrame(results_params)
params_df.to_csv('polynomial_bootstrap_params.csv', index=False)

# Вывод сводной таблицы
print("\nРезультаты полиномиальной регрессии с .632+ бутстрапом")
print("=" * 90)
print(results_df.to_string(index=False, float_format='%.6f'))
print("=" * 90)
print(f"\nМетрики сохранены: polynomial_bootstrap_results.csv")
print(f"Параметры моделей сохранены: polynomial_bootstrap_params.csv")
print(f"\nВсего протестировано: {len(results_metrics)} датасетов")


Прогресс:   0%|                      | 2/540 [02:05<9:21:04, 62.57s/it, deg=1, noise=low, sel_deg=1]




KeyboardInterrupt: 

In [21]:
import pandas as pd
import numpy as np

# Загрузка одного датасета для проверки
degree_true = 6
noise_level = 'moderate'
seed_idx = 0

df = pd.read_csv(f'../datasets/synthetic/synthetic_datasets_with_coeffs/noise_{noise_level}_deg{degree_true}.csv')
seed = df['seed'].unique()[seed_idx]
subset = df[df['seed'] == seed]

x_train = subset['x'].values
y_train = subset['y_noisy'].values

print(f"Тестовый датасет: степень={degree_true}, шум={noise_level}, seed={seed}")
print(f"Размер выборки: {len(x_train)} точек")
print(f"x range: [{x_train.min():.2f}, {x_train.max():.2f}]")
print(f"y range: [{y_train.min():.2f}, {y_train.max():.2f}]")

# Истинные коэффициенты
coeffs_true = [subset[f'coeff_{i}'].iloc[0] for i in range(degree_true + 1)]
print(f"Истинные коэффициенты: {coeffs_true}")

# Подбор модели через .632+ бутстрап
print("\nЗапуск .632+ бутстрапа...")
best_degree, best_c1, best_c2 = full_grid_bootstrap_selection(
    x_train, y_train, 
    degrees=(1, 2, 3, 4, 5, 6),
    B=200,  # уменьшено для быстрой проверки
    random_state=int(seed)
)

print(f"\nРезультаты подбора:")
print(f"  Выбранная степень: {best_degree} (истинная: {degree_true})")
print(f"  Лучшие гиперпараметры: c1={best_c1:.3f}, c2={best_c2:.3f}")

# Оценка коэффициентов
coef_est, y_pred = mixed_norm_polyfit(x_train, y_train, best_c1, best_c2, best_degree)
print(f"  Оценённые коэффициенты: {coef_est}")

# Расчёт метрик
g_true = lambda x, c=coeffs_true: sum(c[i] * x**i for i in range(len(c)))
g_est = lambda x, c=coef_est: sum(c[i] * x**i for i in range(len(c)))

imse_val = imse(g_est, g_true)
imae_val = imae(g_est, g_true)
maxerr_val = maxerr(g_est, g_true)

print(f"\nМетрики качества:")
print(f"  IMSE:  {imse_val:.6f}")
print(f"  IMAE:  {imae_val:.6f}")
print(f"  MaxErr: {maxerr_val:.6f}")

print("\n✅ Пайплайн работает корректно!")

Тестовый датасет: степень=6, шум=moderate, seed=331
Размер выборки: 30 точек
x range: [0.10, 0.99]
y range: [13.86, 25.72]
Истинные коэффициенты: [np.float64(18.743888468071063), np.float64(7.87194406650472), np.float64(5.328263768556655), np.float64(-8.4877694056068), np.float64(7.362215111601699), np.float64(-8.330881889075641), np.float64(-8.74978598554186)]

Запуск .632+ бутстрапа...

Результаты подбора:
  Выбранная степень: 3 (истинная: 6)
  Лучшие гиперпараметры: c1=0.000, c2=1.000
  Оценённые коэффициенты: [ 20.57458412  -8.02805452  52.61583619 -50.52521165]

Метрики качества:
  IMSE:  0.226706
  IMAE:  0.341991
  MaxErr: 0.390065

✅ Пайплайн работает корректно!
